In [ ]:
%matplotlib inline
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from random import shuffle
import math
import json
import h5py
import inspect
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from mpl_toolkits import mplot3d
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

In [2]:
# You may have to change this line given your layout of directories
cur_dir = './drive/My Drive/Diffusion_S1/DNN/'
data = pd.read_hdf(cur_dir + 'output_files/strack_events.h5', '/RECO/Events')

NameError: ignored

In [ ]:
#The lines below take quite a while to run, so I hard-coded the values
#below to avoid wasting that time

#x_all, y_all, z_all = [], [], []
#for event in data.event.unique():
#    my_event = data[data.event == event]
#    image_3d = np.stack((my_event.X, my_event.Y, my_event.Z), axis=1)
#    db = DBSCAN(eps=20, min_samples=30).fit(image_3d)
#    clusters = np.array(db.fit_predict(image_3d))
#    indices = np.array([i for i, x in enumerate(clusters) if x == 0])
#    X_ = np.array(my_event.X)[indices]
#    Y_ = np.array(my_event.Y)[indices]
#    Z_ = np.array(my_event.Z)[indices]
#    x_all.append(max(X_) - min(X_))
#    y_all.append(max(Y_) - min(Y_))
#    z_all.append(max(Z_) - min(Z_))
#x_range, y_range, z_range = max(x_all), max(y_all), math.ceil(max(z_all))

In [ ]:
x_range, y_range, z_range = 280.0, 290.0, 236
X_shape = int(x_range/10) + 2
Y_shape = int(y_range/10) + 1
print(X_shape, Y_shape, x_range, y_range, z_range)

30 30 280.0 290.0 236


In [1]:
# The original data has coordinates in multiples of 10 in X & Y. When you take that and convert
# to a numypy array to make an "image array", it takes a lot of space, the code below, hence,
# divides by that to make the output image smaller
def coord_to_image(X, Y, E):
    image = np.zeros((X_shape, Y_shape))
    for i in range(len(X)):
        if (X[i]%10 == 0) and (Y[i]%10 == 0):
            image[int(Y[i]/10)][int(X[i]/10)] = E[i]
    return image

# After the transformations are applied, the size of the image is already cropped from before, so
# we don't need that division by 10 here
def coord_to_image_plain(X, Y, E):
    image = np.zeros((X_shape, Y_shape))
    for i in range(len(X)):
        image[int(Y[i])][int(X[i])] = E[i]
    return image

def image_to_coord(image):
    Z = np.nonzero(image)[0]
    Y = np.nonzero(image)[1]
    X = np.nonzero(image)[2]
    E = image[np.where(image!=0)]
    return X, Y, Z, E

# This class is used to make various transformations of the original event to get more samples to train on
# The keras' imagedatagenerator method sometimes repeats the original image, which may have lead to 
# over-training so I thought it would be best to code the transformations myself
# The doc line for the method flip_z that says "flip_z" should not be changes as process_data uses
# that information to change the value that the model should predict for that particular
# transformation
class Transform(object):
    def __init__(self, image):
        """init"""
        self.image = image
        return None
    def no_change(self):
        X, Y, Z, E = image_to_coord(self.image)
        Z = Z - min(Z)
        fig = plt.figure()
        return X, Y, Z, E
    def flip_z(self):
        """flip_z"""
        z_image = np.flip(self.image, 0)
        X, Y, Z, E = image_to_coord(z_image)
        Z = Z - min(Z)
        return X, Y, Z, E
    def flip_y(self):
        y_image = np.flip(self.image, 1)
        X, Y, Z, E = image_to_coord(y_image)
        Y = Y - min(Y)
        return X, Y, Z, E
    def flip_x(self):
        x_image = np.flip(self.image, 2)
        X, Y, Z, E = image_to_coord(x_image)
        X = X - min(X)
        return X, Y, Z, E

# This function takes in X,Y,Z,E coordinates and converts that to a numpy array
# It slices the Z coordinates based on a range, so for instance, if your Z coordinates are
# 1.2, 1.4, 2.5, 2.6, 2.7
# and your slice happens to be of "size 1", the coordinates will conver to:
# 1, 1, 2, 2, 2
# z_step determines the width of the slices, the smallest this argument can be is 1
def convert_to_3D(X_, Y_, Z_, E_, z_step):
    my_event = pd.DataFrame({'X' : X_, 'Y' : Y_, 'Z' : Z_, 'E' : E_})
    Z = my_event.Z
    event_image = []
    for z in range(math.floor(min(Z)), math.floor(min(Z)) + z_range + z_step, z_step):
        z_slice = Z[(Z >= z) & (Z < z+z_step)]
        z_slice = z_slice.unique()
        num_z_slices = len(z_slice)
        if num_z_slices == 0:
            X = [0, 0]
            Y = [0, 0]
            E = [0, 0]
            image_x_y = coord_to_image(X, Y, E)
        else:
            z_count = 0
            for each_z in z_slice:
                event_slice = my_event[my_event.Z == each_z]
                X = list(event_slice.X)
                Y = list(event_slice.Y)
                E = list(event_slice.E)
                if z_count == 0:
                    image_x_y = coord_to_image(X, Y, E)
                    z_count += 1
                else:
                    image_x_y += coord_to_image(X, Y, E)
        event_image.append(image_x_y)
    return np.array(event_image), image_x_y.shape[0], image_x_y.shape[1]

# This is only slightly different from above since we do not need to crate z-slices anymore
# The z-coordinates (when this function is called) are already integer slices, so this doesn't
# bother with the slicing
def convert_to_3D_plain(X_, Y_, Z_, E_, N):
    my_event = pd.DataFrame({'X' : X_, 'Y' : Y_, 'Z' : Z_, 'E' : E_})
    event_image = []
    for z in range(math.floor(min(Z_)), max(Z_)):
        X = list(my_event[my_event.Z == z].X)
        Y = list(my_event[my_event.Z == z].Y)
        E = list(my_event[my_event.Z == z].E)
        image_x_y = coord_to_image_plain(X, Y, E)
        event_image.append(image_x_y)
    for z in range(N - (max(Z_) - min(Z_))):
        X = [0, 0]
        Y = [0, 0]
        E = [0, 0]
        image_x_y = coord_to_image(X, Y, E)
        event_image.append(image_x_y)
    return np.array(event_image)

In [ ]:
def process_data(data, z_step, counter_start=0, counter_end=len(data.event.unique())):
    train_data = []
    train_output = []
    counter = counter_start
    for event in data.event.unique()[counter_start:counter_end]:
        print("Onto #" + str(counter))
        my_event = data[data.event == event]
        # this part takes the single track that was identified in prep_data and excludes
        # all the background to get only the single track to be trained by the network
        image_3d = np.stack((my_event.X, my_event.Y, my_event.Z), axis=1)
        db = DBSCAN(eps=20, min_samples=30).fit(image_3d)
        clusters = np.array(db.fit_predict(image_3d))
        indices = np.array([i for i, x in enumerate(clusters) if x == 0])
        # This part centers the entire event to ensure that the information about the 
        # true z coordinates (with the S1 signal) is removed from the event before it is
        # fed to the network
        X_ = np.array(my_event.X)[indices] - min(np.array(my_event.X)[indices])
        Y_ = np.array(my_event.Y)[indices] - min(np.array(my_event.Y)[indices])
        Z_ = np.array(my_event.Z)[indices] - min(np.array(my_event.Z)[indices])
        E_ = np.array(my_event.E)[indices]
        event_image, im_x_y_x, im_x_y_y = convert_to_3D(X_, Y_, Z_, E_,z_step)
        T = Transform(event_image)
        attrs = (getattr(T, name) for name in dir(T))
        methods = filter(inspect.ismethod, attrs)
        for method in methods:
            try:
                if method.__doc__ != "init":
                    try:
                        X, Y, Z, E = method()
                    except:
                        # there are some events that give an error, I haven't looked into them, but 
                        # there's only a handful so I chose to ignore them for the time being
                        print("ERROR!! Event: " + str(event))
                        #fig = plt.figure()
                        #ax = fig.add_subplot(111, projection='3d')
                        #ax.scatter(X, Y, Z)
                    test_image = convert_to_3D_plain(X, Y, Z, E, event_image.shape[0])
                    test_image = np.reshape(test_image, -1)
                    train_data.append(test_image)
                    # this line ensures that the Z coordinate the network is trained on is of the uncropped event
                    # so it gets the true Z position
                    my_event = data[data.event == event]
                    # In the transformation of flipping the event in the z-direction, it's possible that the event has 
                    # more hits at higher z values from the centroid than at lower z values, so when you make the flip
                    # in z direction, the centroid will shift, this code ensures that you take the appropriate centroid value
                    # to train your model on
                    if method.__doc__ == "flip_z":
                        centroid = (max(my_event.Z) - my_event.Z + min(my_event.Z)).mean()
                    else:
                        centroid = my_event.Z.mean()
                    train_output.append([centroid])
                    counter += 1
            except TypeError:
            # cannot handle methods that require arguments
                pass
    return np.array(train_data), np.array(train_output)

In [ ]:
# I did not get around to doing this part and that's why you don't see it being used anywhere below
# This code takes the entire data and then splits it into different .h5 files
# There was a tutorial I was following to be able to train large datasets without
# getting a memory allocation here. Here's that tutorial:
# https://medium.com/@selvam85/how-to-work-with-large-training-dataset-in-google-colab-platform-c3499fc10c24
# Another option that I did not get around to trying was using the .fit_generator method
# If I remember correctly, in process_data, you'd have to change return to a yield
# and yield single events and then feed that to the network below
def write_to_h5(dest_filepath, dest_batch, dataset_name):    
    with h5py.File(dest_filepath, 'a') as f:
        f.create_dataset(dataset_name, dest_batch)

def create_minibatches_h5(data_arr, labels_arr, dest_path, batch_size):
    """To be run only once"""
    t = list(zip(data_arr, labels_arr))
    shuffle(t)
    my_data, my_labels = zip(*t)
    m = len(my_data)
    n_complete_batches = math.ceil(float(m)/batch_size)
    for i in range(n_complete_batches):
        print("Creating file: ", (i+1))
        dest_file = dest_path + str(i + 1) + ".h5" 
        start_pos = i * batch_size
        end_pos = min(start_pos + batch_size, m)
        data_batch = my_data[start_pos:end_pos]
        labels_batch = my_labels[start_pos:end_pos]
        write_to_h5(dest_path + str(i+1), data_batch, "minidata")
        write_to_h5(dest_path + str(i+1), labels_batch, "minilabel")

In [ ]:
# for reasons that are unclear, I could not get more than around 3600 events on a
# single variable. It gives me a memory error but clearly the memory is not getting
# full if you look to top right, but having the data split between two variables
# works. Note that this is a separate issue from training on large data sets
# This is just about being able to load data into memory, the other issue is training
# such a large dataset on the GPU
num_samples = len(data.event.unique())
train_data, train_output = process_data(data, 2, 0, 3600)
train_data_1, train_output_1 = process_data(data, 2, 3600, num_samples)

In [ ]:
para = json.load(open(cur_dir +  "/config.json"))
val_split = 1 - para['train_test_split']
num_epochs = para['epochs']
n_cols = train_data[0].shape[0]
print(n_cols)
model = Sequential()
model.add(Dense(2, activation='relu', input_shape=(n_cols, )))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_absolute_error')
model.summary()

In [ ]:
history = model.fit(train_data, train_output, batch_size=32, validation_split=0.3, epochs=1000, shuffle=True)

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
predict_data, output, predict_x, predict_y = [], [], [], []
for event in range(0, len(train_data), 1):
    predict_data.append(train_data[event])
    output.append(train_output[event])
    predict_x.append(train_x[event])
    predict_y.append(train_y[event])

In [ ]:
predict_output = model.predict(np.array(predict_data))
difference = abs(predict_output - output)

In [ ]:
plt.scatter(output, difference)

In [ ]:
plt.scatter(output, predict_output)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(predict_x, predict_y, difference)